In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Gerekli kütüphaneler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

 1. Veri Setini Tanıma
 2. Eksik Değer Analizi
 3. Veri Görselleştirme (EDA)
 4. Veri Ön İşleme
 5. Feature Engineering
 6. Model Kurma
 7. Model Değerlendirme

 ## 1. Veri Setini Tanıma ##

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
train["Survived"].value_counts(normalize=True)

In [ ]:
train["Sex"].value_counts()

In [ ]:
train["Pclass"].value_counts()

In [ ]:
train["Embarked"].value_counts()

Age sütununda eksik değerler mevcut

Cabin sütunu büyük oranda boş

Sex ve Pclass hayatta kalmayı etkileyebilir

Survived sınıfları tamamen dengeli değil

## 2. Eksik Değer Analizi ##

In [ ]:
train.isnull().sum()

In [ ]:
(train.isnull().sum() / len(train)) * 100

In [ ]:
sns.heatmap(train.isnull(), cbar=False)

Age → doldurulacak (medyan)

Embarked → doldurulacak (mode)

Cabin → silinecek

PassengerId → target dışı, modelde kullanılmayacak

## 3. Veri Görselleştirme (EDA) ##

In [ ]:
sns.countplot(x="Survived", data=train)
# Sınıflar dengeli mi?

In [ ]:
sns.countplot(x="Sex", hue="Survived", data=train)
# Kadınların hayatta kalma oranı çok daha yüksek. Sex kesinlikle modele girmeli

In [ ]:
sns.countplot(x="Pclass", hue="Survived", data=train)
# 1.sınıf → yüksek hayatta kalma
# 3.sınıf → düşük hayatta kalma

In [ ]:
sns.histplot(train["Age"], bins=30, kde=True)

In [ ]:
sns.histplot(data=train, x="Age", hue="Survived", bins=30)
# Çocuklar daha yüksek hayatta kalma oranına sahip

In [ ]:
sns.histplot(data=train, x="Fare", hue="Survived", bins=30)
# Yüksek ücret ödeyenler daha çok hayatta kalmış

In [ ]:
sns.countplot(x="Embarked", hue="Survived", data=train)
# C limanı daha yüksek hayatta kalma oranına sahip

In [ ]:
sns.countplot(x="SibSp", hue="Survived", data=train)

In [ ]:
sns.countplot(x="Parch", hue="Survived", data=train)
# Yalnız veya çok kalabalık aileler dezavantajlı

Sex → çok güçlü değişken

Pclass → çok güçlü değişken

Age → önemli ama eksik değer stratejisi şart

Fare → ölçekleme ve outlier kontrolü gerekebilir

Cabin → bu haliyle anlamsız

Aile bilgileri → feature engineering için uygun

## 4. Veri Ön İşleme ##

In [ ]:
train = train.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
test  = test.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
# Kullanılmayacak Sütunları Kaldırma

In [ ]:
train

In [ ]:
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"]  = test["Age"].fillna(test["Age"].median())
# Eksik Değerleri Doldurma

train


In [ ]:
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])
test["Embarked"]  = test["Embarked"].fillna(test["Embarked"].mode()[0])
# Embarked – En Sık Görülen Değer (Mode)

train

In [ ]:
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

In [ ]:
# Kategorik Değişkenleri Sayısala Çevirme
# Sex – Label Encoding
train["Sex"] = train["Sex"].map({"male": 0, "female": 1})
test["Sex"]  = test["Sex"].map({"male": 0, "female": 1})

In [ ]:
# Embarked – One-Hot Encoding
train = pd.get_dummies(train, columns=["Embarked"], drop_first=True)
test  = pd.get_dummies(test, columns=["Embarked"], drop_first=True)

In [ ]:
train, test = train.align(test, join="left", axis=1, fill_value=0)
# Train ve Test Sütunlarını Eşitleme

In [ ]:
train.isnull().sum()

## 6. Model Kurma ##

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### Logistic Regression ###

In [ ]:
X = train.drop("Survived", axis=1)
y = train["Survived"]
train

In [ ]:
X.isnull().sum()

In [ ]:
# Train – Test Ayrımı %80 eğitim, %20 test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Logistic Regression Modelini Kurma
log_model = LogisticRegression(max_iter=1000)

# Modeli Eğitme
log_model.fit(X_train, y_train)

# Tahmin Yapma
y_pred = log_model.predict(X_test)

In [65]:
# Model Performansını Ölçme
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8100558659217877


In [66]:
confusion_matrix(y_test, y_pred)

array([[90, 15],
       [19, 55]])

In [67]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

